In [0]:
!pip install geopandas

In [0]:
import numpy as np

import pandas as pd
import geopandas as gpd

import psycopg2
import osgeo.ogr
import matplotlib.pyplot as plt

import datetime as dt

import folium

In [3]:
from google.colab import drive
drive.mount('/content/drive')

base_dir = "/content/drive/My Drive/PSDA/" 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
connection = psycopg2.connect(host="193.196.37.97", database="postgres",user="postgres", password="SDAPraktikum2020")
cursor = connection.cursor()  

In [0]:
#stations_data = gpd.read_postgis('SELECT * FROM air_temperature_stations', connection, geom_col='geom_station')
stations = gpd.read_postgis('SELECT stations_id, stationsname, bundesland, geom_station FROM air_temperature_stations', connection, geom_col='geom_station', index_col='stations_id')

In [0]:
data = gpd.read_postgis('SELECT * FROM air_temperature_stations INNER JOIN air_temperature_values ON air_temperature_stations.stations_id=air_temperature_values.stations_id', connection, geom_col='geom_station')

In [0]:
agg_data = data.groupby('stationsname').mean()
agg_data = stations.merge(agg_data, on ='stationsname')

In [0]:
# get all Kreis (Landkreis + Stadtkreis) Shapes
kreis_spatial = gpd.read_file(base_dir + 'landkreise-in-germany.geojson')
kreis_spatial

In [0]:
# get population for landkreise
landkreis_population = pd.read_csv(base_dir + 'landkreise.csv', thousands='.')
landkreis_population = landkreis_population.rename(columns={"KrS": "cca_2"})
landkreis_population['cca_2'] = landkreis_population['cca_2'].astype(str)
landkreis_population['cca_2'] = landkreis_population['cca_2'].apply(lambda x: x.zfill(5))

In [0]:
# get population for stadtkreise
stadtkreis_population = pd.read_csv(base_dir + 'stadtkreise.csv', thousands='.')
stadtkreis_population = stadtkreis_population.rename(columns={'2009':'Einw', 'Kreisfreie Stadt':'name_2'})

In [0]:
# firstly, merge all shapes data with landkreis population dataa
kreis_data = kreis_spatial.merge(landkreis_population, on = 'cca_2', how = 'left')

In [0]:
# secondly, merge resulting data with stadtkreis population data
kreis_data = kreis_data.merge(stadtkreis_population, on = 'name_2', how = 'left')
# select relevant columns
kreis_data = kreis_data[['name_2','type_2','geometry','BevD','Einw_x','Einw_y']]
# replace NAs
kreis_data['Einw_x'] = kreis_data['Einw_x'].fillna(0)
kreis_data['Einw_y'] = kreis_data['Einw_y'].fillna(0)
# Combine landkreis and stadtkreis population columns into one column
kreis_data['Einw'] = kreis_data['Einw_x'] + kreis_data['Einw_y']

In [0]:
# add Stadtkreis/Landkreis to weather stations data
kreise = []
for index, row in agg_data.iterrows():  
  kreis = kreis_data[kreis_data['geometry'].contains(row['geom_station'])]
  if len(kreis) > 0:
    kreise.append(kreis.name_2.iloc[0])
  else:
    kreise.append('')
agg_data['Landkreis'] = kreise

In [0]:
# store county data to csv
kreis_data[['name_2','type_2','geometry','BevD','Einw']].to_csv(base_dir + 'county_data')

In [0]:
# store stations data with average temperature and corresponding county to csv
agg_data.to_csv(base_dir + 'stations+counties.csv')